위조범 네트워크와 전문가 네트워크가 바로 GAN이다. 두 네트워크는 상대를 이기기 위해 훈련합니다. GAN 네트워크 2개는 다음과 같습니다.

- 생성자 네트워크: 랜덤 벡터를 입력으로 받아 합성된 이미지로 디코딩합니다.

- 판별자 네트워크: 이미지를 입력으로 받아 훈련 세트에서 온 이미지인지 생성자 네트워크가 만든 이미지인지 판별한다.

생성자 네트워크는 판별자 네트워크를 속이도록 훈련합니다. 훈련이 계속될수록 점점 더 실제와 같은 이미지를 생성한다. 실제 이미지와 구분할 수 없는 인공지능 이미지를 만들어 판별자네트워크가 두 이미지를 동일하게 보도록 만듭니다. 


한편 판별자 네트워크는 생성된 이미지가 실제인지 판별하는 기준을 설정하면서 생성자의 능력 향상에 적응해 갑니다. 훈련이 끝나면 생성자는 입력 공간에 있는 어떤 포인트를 그럴듯한 이미지로 변환합니다. VAE와 달리 잠재 공간은 의미 있는 구조를 보장하지 않습니다. 특히 이 공간은 연속적이지 않습니다.

- 놀랍게도 GAN은 최적화의 최솟값이 고정되지 않았다. 이 책에서 다루는 어떤 훈련 설정과도 다릅니다. GAN에서는 언덕을 내려오는 매 단계가 전체 공간을 조금씩 바꿉니다. 최적화 과정이 최솟값을 찾는 것이 아니라 두 힘 간의 평형점을 찾는 다이나믹 시스템이다. 이런 이유로 GAN은 훈련이 어렵다. GAN을 만들려면 모델 구조와 훈련 파라미터를 주의 깊게 많이 조정해야 한다.

## 1. GAN 구현 방법

- gan 구조는 다음과 같다.

1. generator 네트워크는 (latent_dim,) 크기 벡터를 (32, 32, 3) 크기의 이미지로 매핑합니다.

2. discriminator 네트워크는 (32, 32, 3) 크기 이미지가 진짜일 확률을 추정하여 이진 값으로 매핑합니다.

3. 생성자와 판별자를 연결하는 gan 네트워크를 만듭니다. gan(x) = discriminator(generator(x))입니다. 이 네트워크는 잠재 공간의 벡터를 판별자의 평가로 미팽합니다. 즉 판별자는 생성자가 잠재 공간의 벡터를 디코딩한 것이 얼마나 현실적인지 판단합니다.

4. 진짜 / 가짜 레이블과 함께 진짜 이미지와 가짜 이미지 샘플을 사용하여 판별자를 훈련합니다. 일반적인 분류 모델을 훈련하는 것과 동일합니다.

5. 생성자를 훈련하려면 gan 모델 손실에 대한 생성자 가중치 그래디언트를 사용합니다. 이 말은 매 단계마다 생성자에 의해 디코딩된 이미지를 판별자가 "진짜'로 분류하는 방향으로 생성자의 가중치를 이동합니다. 다른 말로 판별자를 속이도록 생성자를 훈련합니다.

## 2. 훈련 방법

GAN을 훈련하는 것은 어렵습니다. 이런 기법들은 이론에 바탕을 둔 지침이라기보다는 경험에 의해서 발견되었다.

- 생성자 마지막 활성화로 다른 종류의 모델에서널리 사용하는 sigmoid 대신 tanh를 사용한다.

- 균등 분포가 아니라 정규 분포를 사용하여 잠재 공간에서 포인트를 샘플링합니다.

- 무작위성은 모델을 견고하게 만듭니다. GAN 훈련은 동적 평형을 만들기 때문에 여러 방식으로 갇힐 가능성이 높다. 훈련하는 동안 무작위성을 주입하면 이를 방지하는데 도움이 됩니다. 무작위성은 두 가지 방법으로 주입합니다. 편발자에 드롭아웃을 사용하거나 판별자를 위해 레이블에 랜덤 노이즈를 추가합니다.

- 희소한 그래디언트는 GAN 훈련을 방지할 수 있습니다. 딥러닝에서 희소는 종종 바람직한 현상이지만 GAN에서는 그렇지 않습니다. 그래디언트를 희소하게 만들 수 있는 것은  최대 풀링과 ReLU 두 가지입니다. 최대 풀링 대신 스트라이드 합성곱을 사용하여 다운 샘플링을 하는 것이 좋습니다. 또 Relu 대신 LeakyReLU 층을 사용하세요. ReLU와 비슷하지만 음수의 활성화 값을 조금 허용하기 때문에 희소가 조금 완화됩니다.

- 생성자 픽셀 공간을 균일하게 다루지 못하여 생성된 이미지에서 체스판 모양이 종종 나타납니다. 이를 해결하기 위해 생성자와 판별자에서 스트라이드 ConvTranspose나 Conv2D를 사용할 때 스트라이드 크기로 나눠질 수 있는 커널 크기를 사용합니다.

## 3. 생성자

먼저 벡터를 후보 이미지로 변환하는 generator 모델을 만들어본다. GAN에서 발생하는 많은 문제 중 하나는 생성자가 노이즈 같은 이미지를 생성하는 데서 멈추는 것이다. 

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = False

config.gpu_options.per_process_gpu_memory_fraction = 0.1
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
import keras
from keras import layers

In [7]:
latent_dim = 32
height = 32
width =32
channels = 3

In [5]:
generator_input = keras.Input(shape=(latent_dim,))

# 16 * 16 * 128
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# 16, 16, 128
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# transpose convolution - 32, 32, 256
x = layers.Convolution2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

# convolution
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

## 4. 판별자

In [9]:
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

In [12]:
# clipping과 학습률 감쇠
discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

## 5. 적대적 네트워크

- 생성자와 판별자를 연결하여 GAN을 설정한다. 훈련에 사용되는 타깃 레이블을 항상 진짜 이미지입니다. gan을 훈련하는 것은 discriminaotr가 가짜 이미지를 보았을 때 진짜라고 예측하기 위해 generator의 가중치를 업데이트한다.

In [13]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim, ))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

## 6. DCGAN 훈련 방법

- 훈련 절차


1. 잠재 공간에서 무작위로 포인트를 뽑습니다. (랜덤 노이즈)
2. 이 랜덤 노이즈를 사용하여 generator에서 이미지를 생성합니다.
3. 생성된 이미지와 진짜 이미지를 섞습니다.
4. 진짜와 가짜가 석인 이미지와 이에 대응하는 타깃을 사용하여 discriminator를 훈련합니다. 타깃은 진짜 또는 가짜입니다.
5. 잠재 공간에서 무작위로 새로운 포인트를 뽑습니다.
6. 이 랜덤 벡터를 사용하여 gan을 훈련합니다. 모든 타깃은 '진짜'로 설정합니다. 판별자가 생성된 이미지를 모두 '진짜 이미지'라고 예측하도록 생성자의 가중치를 업데이트합니다. 결국 생성자는 판별자를 속이도록 훈련합니다.